In [1]:
# Ensure all required packages are installed
!pip install datasets transformers rouge_score nltk

# Import necessary libraries
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import re

# 1. Load the CNN/Daily Mail dataset
dataset = load_dataset("abisee/cnn_dailymail", "3.0.0")

# Split the dataset
train_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']

print("Dataset loaded successfully!")

# 2. Preprocess the text
def clean_text(text):
    """Function to clean and normalize text."""
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    return text.lower()

# Apply text cleaning to training data
train_articles = [clean_text(example['article']) for example in train_data]
train_summaries = [clean_text(example['highlights']) for example in train_data]

# Example of cleaned text
print("Cleaned Article Example:", train_articles[0])
print("Cleaned Summary Example:", train_summaries[0])

# 3. Extractive summarization
print("Starting extractive summarization...")
extractive_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Perform summarization on a test article
test_article = test_data[0]['article']
extractive_summary = extractive_summarizer(test_article, max_length=50, min_length=25, truncation=True)
extractive_summary_text = extractive_summary[0]['summary_text']

print("Extractive Summary:", extractive_summary_text)

# 4. Abstractive summarization
print("Starting abstractive summarization...")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

inputs = tokenizer(test_article, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
abstractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Abstractive Summary:", abstractive_summary)

# 5. Evaluation metrics
print("Evaluating summaries...")
reference = test_data[0]['highlights']

# ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores_extractive = scorer.score(reference, extractive_summary_text)
rouge_scores_abstractive = scorer.score(reference, abstractive_summary)

# BLEU score
bleu_score_abstractive = sentence_bleu([reference.split()], abstractive_summary.split())

# Display results
print("ROUGE Scores (Extractive):", rouge_scores_extractive)
print("ROUGE Scores (Abstractive):", rouge_scores_abstractive)
print("BLEU Score (Abstractive):", bleu_score_abstractive)

# 6. Save results to file
with open("summary_results.txt", "w") as f:
    f.write(f"Extractive Summary:\n{extractive_summary_text}\n")
    f.write(f"Abstractive Summary:\n{abstractive_summary}\n")
    f.write(f"ROUGE Scores (Extractive): {rouge_scores_extractive}\n")
    f.write(f"ROUGE Scores (Abstractive): {rouge_scores_abstractive}\n")
    f.write(f"BLEU Score (Abstractive): {bleu_score_abstractive}\n")

print("Summarization and evaluation completed! Results saved in 'summary_results.txt'.")


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=6170136c3a121540320502e3239e3d324b59a347c29dddf2a42b9ce19c052f55
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Extractive Summary: Palestinian Authority becomes 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body.
Starting abstractive summarization...
Abstractive Summary: The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body.
Evaluating summaries...
ROUGE Scores (Extractive): {'rouge1': Score(precision=0.5428571428571428, recall=0.5588235294117647, fmeasure=0.5507246376811593), 'rouge2': Score(precision=0.38235294117647056, recall=0.3939393939393939, fmeasure=0.38805970149253727), 'rougeL': Score(precision=0.4857142857142857, recall=0.5, fmeasure=0.49275362318840576)}
ROUGE Scores (Abstractive): {'rouge1': Score(precision=0.5135135135135